In [6]:
from datasets import load_dataset
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import LanceDB
import lancedb
my_emb = OllamaEmbeddings(model="tinyllama")
db = lancedb.connect("data-guten/lance.db",)

In [7]:
# Load the Hugging Face dataset
print("Loading dataset...")
dataset = load_dataset("kmfoda/booksum", split="test")
# Preprocess the text
texts = [item['chapter'] for item in dataset if item['chapter']]
print("dataset loaded")

Loading dataset...


In [10]:
vectorstore = LanceDB.from_texts(texts = texts,embedding=my_emb,connection=db)


KeyboardInterrupt: 

In [ ]:
retriever=vectorstore.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# LLM
ollama_llm = "llama2"
model = ChatOllama(model=ollama_llm)
print("Created Ollama model")

In [3]:
# def func(_dict):
#     return list(map(lambda x:x.page_content,_dict["context"]))
# RAG chain
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)
print("chain done")

Loading dataset...


In [ ]:
table = db.create_table("rag",data=[
            {
                "vector": my_emb.embed_query("Hello World"),
                "text": "Hello World",
                "id": "1",
            }
        ],
        mode="overwrite",)

In [ ]:
vectorstore = LanceDB(connection=table, embedding=my_emb)
retriever = vectorstore.as_retriever()

In [2]:
chain.invoke('say hello')

"Hello Harrison! It's great to see you here. How are you doing today?"